
 #### Use this workbook to predict how much you should receive and you can compare against Agent/Bank statements
 
 #### There are a few parameters to be updated 
 
 * first update the from and to (match it with statement from agent)
 * update second table for maintenance table and water usage. Make sure the length of array the same.



```python

mydf[(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)]
```
 
 

In [16]:
fr = '2019-10-07'   # enter from date    
to = '2019-10-18'   # enter to date 

In [17]:
import pandas as pd
import numpy as np
import os
import glob


# pick up all offset files
os.chdir("/Users/ronny/CBAStatements/Offset")
extension = 'csv'
offset_file = [i for i in glob.glob('*.{}'.format(extension))]
offset_csv = pd.concat([pd.read_csv(f, header=None) for f in offset_file ])

# pick up all card statements
os.chdir("/Users/ronny/CBAStatements/Card")
card_file = [i for i in glob.glob('*.{}'.format(extension))]
card_csv = pd.concat([pd.read_csv(f, header=None) for f in card_file ])



df = pd.concat([offset_csv,card_csv],ignore_index=True)
df = df.drop([3],axis=1)
df.columns = ['Date','Amount','Description']
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)



In [18]:
# update maintenance table manually since they are adhoc 


# maintenance table/other for The Ponds
mydate = ('2019-08-26','2019-08-26')
myamt = (220,198)
mydesc = ('Downlight','Oven Installation')

df_maintenance = pd.DataFrame({'Wk Beg':mydate, 'Desc':mydesc , 'Expected':myamt})


# maintenance/other table for Wolli Creek
mydate = ('2019-09-16','2019-09-16','2019-09-16')
myamt = (236.36,136.36,29.23)
mydesc = ('Letting Fee','Advertising Fee','Lease Prep Fee')

df_maintenance2 = pd.DataFrame({'Wk Beg':mydate, 'Desc':mydesc , 'Expected':myamt})

# usage table
mydate = ('2019-05-26','2019-08-26','2019-09-16','2019-09-16')
myamt = (205,289.68,-205,-282.66)
mydesc = ('June Qtr Usage','Sept Qtr Usage','Paid - Usage 14/2 to 9/5','Paid - Usage 10/5 to 13/8')

df_water_usage = pd.DataFrame({'Wk Beg':mydate, 'Desc':mydesc , 'Expected':myamt})


##### This workbook will also download the bank statement (CBA) using CSV.  
##### See below information, it will group description related to property.   
##### Need to review them as required if there are changes in the descriptions.  

In [19]:
import pandas as pd

mystr = ['Harcourts','Hills Prestige','Hillside','Slick','Carena','Netbank Admiral','repay 778047926','repay 778047934','repay 876891821']
mygroup = ['The Ponds Rental','The Ponds Rental','The Ponds Rental','Wolli Rental','Wolli Rental','The Ponds WBC','Wolli CBA','The Ponds CBA1','The Ponds CBA2']

mydf = df.loc[0:1]
mydf['Group'] = 'Other'

for i in range(len(mystr)):
        data = df[df['Description'].str.contains(mystr[i],case=False)]
        data['Group'] = mygroup[i]
        mydf = mydf.append(data)

mydf = mydf.drop(mydf.index[0:2])

mydf.columns = ['Wk Beg', 'Amount', 'Description', 'Group']


/Users/ronny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ronny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [20]:
bank_thepond = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='The Ponds Rental')].sum()
bank_wolli = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='Wolli Rental')].sum()
bank_cbawolli = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='Wolli CBA')].sum()
bank_wbc = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='The Ponds WBC')].sum()
bank_cba1 = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='The Ponds CBA1')].sum()
bank_cba2 = mydf['Amount'][(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='The Ponds CBA2')].sum()

In [21]:
# create different tables for predicting expected amounts

mydate = pd.date_range('2018-05-21','2029-09-23', freq='7D')
myamt = np.repeat(650,len(mydate))
mydesc = 'The Ponds - Rental'
df_rental = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})


# wolli creek rental new tenants starting 24/9 at 520

mydate = pd.date_range('2018-02-12','2019-08-18', freq='7D')
myamt = np.repeat(560,len(mydate))
mydesc = 'Wolli Rental - Tenant 1'
df_rental1_1 = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})


mydate = pd.date_range('2019-09-24','2029-09-23', freq='7D')
myamt = np.repeat(520,len(mydate))
mydesc = 'Wolli Rental - Tenant 2'
df_rental1_2 = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})

df_rental2 = df_rental1_1.append(df_rental1_2)



# others
mydate = pd.date_range('2018-05-15','2020-05-01', freq='QS-MAY') + pd.DateOffset(days=15)
myamt = np.repeat(383,len(mydate))
mydesc = 'Council'
df_council = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})

mydate = pd.date_range('2018-05-15','2020-06-01', freq='QS-MAY') + pd.DateOffset(days=15)
myamt = np.repeat(172.04,len(mydate))
mydesc = 'Water'
df_water = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})

mydate = pd.date_range('2018-07-04','2020-06-30', freq='7D')
myamt = np.repeat(889.85,len(mydate))
mydesc = 'Interest WBC'
df_wbc = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})

mydate = pd.date_range('2018-05-07','2020-06-30', freq='MS') + pd.DateOffset(days=9)
myamt = np.repeat(307.97,len(mydate))
mydesc = 'Interest CBA1'
df_cba1 = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})

mydate = pd.date_range('2018-05-07','2020-06-30', freq='MS') + pd.DateOffset(days=7)
myamt = np.repeat(186,len(mydate))
mydesc = 'Interest CBA2'
df_cba2 = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})


mydate = pd.date_range('2018-05-07','2020-06-30', freq='MS') + pd.DateOffset(days=7)
myamt = np.repeat(1145.19,len(mydate))
mydesc = 'Interest Wolli'
df_wolli = pd.DataFrame ({'Wk Beg':mydate, 'Desc':mydesc,'Expected':myamt})


In [22]:
rental = df_rental['Expected'][(df_rental['Wk Beg']>fr)&(df_rental['Wk Beg']<=to)].sum()
rental2 = df_rental2['Expected'][(df_rental2['Wk Beg']>=fr)&(df_rental2['Wk Beg']<to)].sum()
council = df_council['Expected'][(df_council['Wk Beg']>fr)&(df_council['Wk Beg']<=to)].sum()
water = df_water['Expected'][(df_water['Wk Beg']>fr)&(df_water['Wk Beg']<=to)].sum()
water_usage = df_water_usage['Expected'][(df_water_usage['Wk Beg']>fr)&(df_water_usage['Wk Beg']<=to)].sum()
maintenance = df_maintenance['Expected'][(df_maintenance['Wk Beg']>fr)&(df_maintenance['Wk Beg']<=to)].sum()
maintenance2 = df_maintenance2['Expected'][(df_maintenance2['Wk Beg']>fr)&(df_maintenance2['Wk Beg']<=to)].sum()
wbc = df_wbc['Expected'][(df_wbc['Wk Beg']>fr)&(df_wbc['Wk Beg']<=to)].sum()
cba1 = df_cba1['Expected'][(df_cba1['Wk Beg']>fr)&(df_cba1['Wk Beg']<=to)].sum()
cba2 = df_cba2['Expected'][(df_cba2['Wk Beg']>fr)&(df_cba2['Wk Beg']<=to)].sum()
cbawolli = df_wolli['Expected'][(df_wolli['Wk Beg']>fr)&(df_wolli['Wk Beg']<=to)].sum()
commission = rental * 5.5 /100
commission2 = rental2 * 4 /100
sundry = 2.75

In [23]:
pd.DataFrame({'From': fr,
              'To': to,
              '# Days': pd.to_datetime(to) - pd.to_datetime(fr),
              'Rental':rental,
              'Commission':commission,
              'Sundry':sundry,
              'Water':water,
              'WaterUsge':water_usage,
              'Council':council,
              'Maint':maintenance,
              'Expected in Bank':rental-commission-sundry-water-water_usage-council-maintenance,
              'From Agent' : bank_thepond,
              'Check Diff' : bank_thepond - (rental-commission-sundry-water-water_usage-council-maintenance),
              'WBC': wbc,
              'CBA1': round(cba1,0),
              'CBA2': round(cba2,0),
              'Investment Loss': round(rental-commission-sundry-water-council-maintenance - wbc - cba1 - cba2,0)
             }, index=['The Ponds Calc >>'])

,From,To,# Days,Rental,Commission,Sundry,Water,WaterUsge,Council,Maint,Expected in Bank,From Agent,Check Diff,WBC,CBA1,CBA2,Investment Loss
The Ponds Calc >>,2019-10-07,2019-10-18,11 days,650,35.75,2.75,0.0,0.0,0,0,611.5,0.0,-611.5,1779.7,308.0,186,-1662.0


In [24]:
myreport = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':rental - commission - sundry - maintenance - water - water_usage - council,
              'Received':bank_thepond,
              'Diff': bank_thepond - (rental - commission - sundry - maintenance - water - water_usage - council) 
             }, index=['1. Rental Collection : The Ponds >>'])

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':rental,
              'Received':0,
              'Diff': 0
             }, index=['Weekly Rental'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': commission + sundry,
              'Received':0,
              'Diff': 0
             }, index=[' Commission & Sundry'])

myreport = myreport.append(tmp)

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': maintenance,
              'Received':0,
              'Diff': 0
             }, index=[' Maintenance'])

myreport = myreport.append(tmp)

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': water + water_usage,
              'Received':0,
              'Diff': 0
             }, index=[' Water ~$171 + usage'])

myreport = myreport.append(tmp)

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': council,
              'Received':0,
              'Diff': 0
             }, index=[' Council'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':wbc + cba1 + cba2,
              'Received': -(bank_wbc + bank_cba1 + bank_cba2),
              'Diff': (wbc + cba1 + cba2) + (bank_wbc + bank_cba1 + bank_cba2)
             }, index=['2. Bank Interest : The Ponds >>>>>'])

myreport = myreport.append(tmp)

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':wbc,
              'Received': -bank_wbc,
              'Diff': bank_wbc + wbc
             }, index=['WBC'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':cba2,
              'Received':-bank_cba1,
              'Diff': bank_cba1 + cba2
             }, index=['CBA1'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':cba1,
              'Received':-bank_cba2,
              'Diff': bank_cba2 + cba1
             }, index=['CBA2'])

myreport = myreport.append(tmp)

myreport


,From,To,Expected,Received,Diff
1. Rental Collection : The Ponds >>,2019-10-07,2019-10-18,611.50,0.00,-611.50
Weekly Rental,2019-10-07,2019-10-18,650.00,0.00,0.00
Commission & Sundry,2019-10-07,2019-10-18,38.50,0.00,0.00
Maintenance,2019-10-07,2019-10-18,0.00,0.00,0.00
Water ~$171 + usage,2019-10-07,2019-10-18,0.00,0.00,0.00
Council,2019-10-07,2019-10-18,0.00,0.00,0.00
2. Bank Interest : The Ponds >>>>>,2019-10-07,2019-10-18,2273.67,1187.89,1085.78
WBC,2019-10-07,2019-10-18,1779.70,889.85,889.85
CBA1,2019-10-07,2019-10-18,186.00,-0.00,186.00
CBA2,2019-10-07,2019-10-18,307.97,298.04,9.93


In [33]:
myreport = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':rental2 - commission2 - maintenance2,
              'Received':bank_wolli,
              'Diff': bank_wolli-(rental2 - commission2 - maintenance2)
             }, index=['1. Rental Collection : Wolli Creek >>'])

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected':rental2,
              'Received':0,
              'Diff': 0
             }, index=['Weekly Rental'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': commission2,
              'Received':0,
              'Diff': 0
             }, index=[' Commission & Sundry'])

myreport = myreport.append(tmp)

tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': maintenance2,
              'Received':0,
              'Diff': 0
             }, index=[' Maintenance & Other Wolli Creek'])

myreport = myreport.append(tmp)


tmp = pd.DataFrame({'From': fr,
              'To': to,
              'Expected': cbawolli,
              'Received': -bank_cbawolli,
              'Diff': cbawolli + bank_cbawolli
             }, index=['2. Bank Interest : Wolli >>>>>'])

myreport = myreport.append(tmp)


myreport


,From,To,Expected,Received,Diff
1. Rental Collection : Wolli Creek >>,2019-10-07,2019-10-18,998.40,833.40,-165.00
Weekly Rental,2019-10-07,2019-10-18,1040.00,0.00,0.00
Commission & Sundry,2019-10-07,2019-10-18,41.60,0.00,0.00
Maintenance & Other Wolli Creek,2019-10-07,2019-10-18,0.00,0.00,0.00
2. Bank Interest : Wolli >>>>>,2019-10-07,2019-10-18,1145.19,1419.25,-274.06


In [26]:
df_rental2[(df_rental2['Wk Beg']>=fr)&(df_rental2['Wk Beg']<to)]

,Wk Beg,Desc,Expected
2,2019-10-08,Wolli Rental - Tenant 2,520
3,2019-10-15,Wolli Rental - Tenant 2,520


In [27]:
mydf[(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='Wolli Rental')]

,Wk Beg,Amount,Description,Group
2,2019-10-15,833.4,Direct Credit 010842 SLICKPROPEMAP Mid Mnth Rn...,Wolli Rental


In [28]:
mydf[(mydf['Wk Beg']>fr)&(mydf['Wk Beg']<=to)&(mydf.Group=='Wolli CBA')]

,Wk Beg,Amount,Description,Group
9,2019-10-09,-311.00,Loan Repayment LN REPAY 778047926,Wolli CBA
11,2019-10-08,-1108.25,Loan Repayment LN REPAY 778047926,Wolli CBA
